In [10]:

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
import numpy as np

In [63]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
custom_stop_words = ["mr", "ms", "mrs", "said"]  # Example custom additions
stop_words.update(custom_stop_words)
stop_words = list(stop_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [71]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [72]:
import pandas as pd
file_path = "/content/drive/My Drive/nyt_articles_sentiment.csv"

dataframe = pd.read_csv(file_path, delimiter=",")
econdf = dataframe[dataframe["subsection"]== "Economy"]
politicsdf = dataframe[dataframe["subsection"]== "Politics"]

import re

# Function to clean the snippet text
def clean_snippet(text):
    # Remove non-alphabetic characters and punctuation, except spaces
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Remove single-character tokens (like "j", "M")
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Apply the cleaning function to the 'snippet' column
econdf['snippet'] = econdf['snippet'].apply(clean_snippet)
politicsdf['snippet'] = politicsdf['snippet'].apply(clean_snippet)

# Drop rows where the cleaned snippet is empty
econdf = econdf[econdf['snippet'].str.strip() != ""]
politicsdf = politicsdf[politicsdf['snippet'].str.strip() != ""]

dataframe = politicsdf

pd.set_option('display.max_columns',1000)
pd.set_option('display.width',10000)
print(econdf)
print(politicsdf)

<ipython-input-72-066e08b32d77>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  econdf['snippet'] = econdf['snippet'].apply(clean_snippet)


                                                  title                                            snippet                                                url            published_date       section subsection sentiment_label  sentiment_score
58    Federal Reserve Won’t Backpedal on Interest Ra...  Ms Yellen the Fed chairwoman told senators tha...  https://www.nytimes.com/2016/02/12/business/ec...  2016-02-11T19:09:41+0000  Business Day    Economy        positive         0.648025
102   The Cities on the Sunny Side of the American E...  In places like Denver Seattle and Atlanta empl...  https://www.nytimes.com/2016/04/01/business/ec...  2016-03-31T18:54:38+0000  Business Day    Economy        positive         0.403958
104                City by City, Measuring the Recovery  Cities like Denver San Jose Calif and Austin T...  https://www.nytimes.com/interactive/2016/04/01...  2016-03-31T16:50:11+0000  Business Day    Economy        negative         0.823946
114   Joel Kurtzman, Economist o

<ipython-input-72-066e08b32d77>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  politicsdf['snippet'] = politicsdf['snippet'].apply(clean_snippet)


In [73]:
import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, max_df=0.9, min_df=10, stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(dataframe['snippet'])


In [74]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate sentence embeddings
embeddings = model.encode(dataframe['snippet'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/101 [00:00<?, ?it/s]

In [75]:
from sklearn.cluster import DBSCAN

# Perform DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='cosine')
labels = dbscan.fit_predict(embeddings)

# Add cluster labels to the DataFrame
dataframe['cluster'] = labels
print(dataframe)

                                                  title                                            snippet                                                url            published_date section subsection sentiment_label  sentiment_score  cluster
0     An Ice Cream Mogul and His Fellow Burlingtonia...  In liberal enclave fiercely loyal to Bernie Sa...  https://www.nytimes.com/politics/first-draft/2...  2016-01-07T23:04:26+0000    U.S.   Politics        negative         0.757460        0
1     Bill Clinton, Back in Iowa, Refuses to Respond...  Bill Clinton returned to the campaign trail in...  https://www.nytimes.com/politics/first-draft/2...  2016-01-07T22:49:18+0000    U.S.   Politics         neutral         0.789728        0
2               Donald Trump Brings Gridlock to Vermont  Donald Trump brought gridlock to Burlington Vt...  https://www.nytimes.com/politics/first-draft/2...  2016-01-07T22:23:31+0000    U.S.   Politics        negative         0.942191       -1
3     Ad Wars of 201

In [76]:
# Get feature names
feature_names = vectorizer.get_feature_names_out()

# For each cluster (excluding noise: label `-1`)
unique_clusters = set(labels) - {-1}  # Exclude noise
for cluster in unique_clusters:
    # Find indices for the current cluster
    cluster_indices = (dataframe['cluster'] == cluster).values.nonzero()[0]

    # Subset the TF-IDF matrix for this cluster
    cluster_texts = tfidf_matrix[cluster_indices]

    # Sum term frequencies and get top terms
    top_terms_indices = np.asarray(cluster_texts.sum(axis=0)).flatten().argsort()[::-1][:10]
    top_terms = [feature_names[i] for i in top_terms_indices]

    print(f"Cluster {cluster}: {top_terms}")

Cluster 0: ['president', 'trump', 'donald', 'house', 'former', 'would', 'new', 'democrats', 'trumps', 'republicans']
Cluster 1: ['campaign', 'amid', 'trump', 'brought', 'even', 'government', 'reporter', 'incident', 'charge', 'concerns']
Cluster 2: ['education', 'secretary', 'schools', 'private', 'progress', 'school', 'made', 'tough', 'president', 'away']
Cluster 3: ['john', 'staff', 'chief', 'needs', 'white', 'willing', 'trump', 'house', 'step', 'military']
Cluster 4: ['reserve', 'federal', 'economy', 'economic', 'coronavirus', 'jackson', 'numbers', 'current', 'financial', 'issues']
Cluster 5: ['afghanistan', 'veterans', 'summer', 'got', 'biden', 'three', 'year', 'president', 'fought', 'prominent']
Cluster 6: ['governor', 'gov', 'virginia', 'sexual', 'assault', 'past', 'demands', 'school', 'medical', 'months']


In [77]:
econcat = {"0": 1.3, "1": 1.1, "-1": 0.7}
polcat = {"0": 0.8, "1":0.6, "2":0.4, "3": 0.7, "4": 1.3, "5": 0.9, "6": 0.5, "-1": 0.3}

dataframe['scaling_score'] = dataframe['cluster'].astype(str).map(lambda x: polcat.get(x))
# dataframe['scaling_score'] = dataframe['cluster'].astype(str).map(lambda x: econcat.get(x))
dataframe['scaled_score'] = dataframe['sentiment_score'] * dataframe['scaling_score']
print(dataframe)
save_directory = "/content/drive/My Drive/nyt_articles_pol_sentiment.csv"
# save_directory = "/content/drive/My Drive/nyt_articles_econ_sentiment.csv"
dataframe.to_csv(save_directory, index=False)

                                                  title                                            snippet                                                url            published_date section subsection sentiment_label  sentiment_score  cluster  scaling_score  scaled_score
0     An Ice Cream Mogul and His Fellow Burlingtonia...  In liberal enclave fiercely loyal to Bernie Sa...  https://www.nytimes.com/politics/first-draft/2...  2016-01-07T23:04:26+0000    U.S.   Politics        negative         0.757460        0            0.8      0.605968
1     Bill Clinton, Back in Iowa, Refuses to Respond...  Bill Clinton returned to the campaign trail in...  https://www.nytimes.com/politics/first-draft/2...  2016-01-07T22:49:18+0000    U.S.   Politics         neutral         0.789728        0            0.8      0.631782
2               Donald Trump Brings Gridlock to Vermont  Donald Trump brought gridlock to Burlington Vt...  https://www.nytimes.com/politics/first-draft/2...  2016-01-07T22:23:31

In [78]:
econpath = "/content/drive/My Drive/nyt_articles_econ_sentiment.csv"
econdf = pd.read_csv(econpath, delimiter=",")

polipath = "/content/drive/My Drive/nyt_articles_pol_sentiment.csv"
politicsdf = pd.read_csv(polipath, delimiter=",")

combined_df = pd.concat([econdf, politicsdf], ignore_index=True)
print(combined_df)

save_directory = "/content/drive/My Drive/nyt_articles_sentiment_scaled.csv"
combined_df.to_csv(save_directory, index=False)

                                                  title                                            snippet                                                url            published_date       section subsection sentiment_label  sentiment_score  cluster  scaling_score  scaled_score
0     Federal Reserve Won’t Backpedal on Interest Ra...  Ms Yellen the Fed chairwoman told senators tha...  https://www.nytimes.com/2016/02/12/business/ec...  2016-02-11T19:09:41+0000  Business Day    Economy        positive         0.648025        0            1.3      0.842433
1     The Cities on the Sunny Side of the American E...  In places like Denver Seattle and Atlanta empl...  https://www.nytimes.com/2016/04/01/business/ec...  2016-03-31T18:54:38+0000  Business Day    Economy        positive         0.403958        0            1.3      0.525146
2                  City by City, Measuring the Recovery  Cities like Denver San Jose Calif and Austin T...  https://www.nytimes.com/interactive/2016/04/01...  2